<h1>Web Scraping Weather with beautiful soup</h1>

- I use beautifulsoup to scrap a weather site
- I extract the 7 day forcast and save it to a csv file

In [1]:
import sqlalchemy as db
#from google.cloud import bigquery
from bs4 import BeautifulSoup
import requests
import datetime
import pandas as pd
import os.path

In [2]:
url_14_days = "https://www.meteomedia.com/ca/tendance-meteo-14-jours/quebec/montreal"
url_env_can = 'https://weather.gc.ca/city/pages/qc-147_metric_e.html'
page_response = requests.get(url_env_can, timeout = 5)
#print(page_response.status_code)
soup = BeautifulSoup(page_response.content, "html.parser")

In [10]:
# week forecast
columns = ['weekday', 'date', 'month', 'max_temp', 'max_sky', 'min_temp', 'min_sky']
wd = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
mnth = {1: 'Janvier', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', \
        9: 'Septembrer', 10: 'October', 11: 'November', 12: 'Decembre'}
now_ = datetime.datetime.now()
date = []
min_c = []
max_c = []
meteo = []
mn = soup.find_all("div", {"class":"div-row div-row4 div-row-data"})
mx = soup.find_all("div", {"class":"div-row div-row2 div-row-data"})
dts = list(soup.find_all("div", {"class":"div-row div-row3 div-row-head"}))
#print(dts)

for i in range(len(dts)):
    if i > 0 : 
        dt = dts[i].get("title").split()
        date.append([dt[0], dt[2], dt[3]])
    else:
        date.append([wd[now_.weekday()], now_.day, mnth[now_.month]])
    #print('min', i,(mn[i].text))
    #print('max', i,(mx)[i].text)
    min_c.append((mn[i].text))
    max_c.append((mx[i].text))

for i in range(len(dts)):
    meteo.append([date[i][0], 
                  date[i][1], 
                  date[i][2].replace('\n', ' '), 
                  max_c[i][1:max_c[0].find("°C")].replace('\n', ' '), 
                  max_c[i][max_c[0].find("°F")+4:-1].replace('\n', ' '), 
                  min_c[i][1:min_c[0].find("°C")].replace('\n', ' '),
                  min_c[i][min_c[0].find("°F")+4:-1].replace('\n', ' ')])

    
# write to a csv file
df = pd.DataFrame(meteo, columns = columns) 
fname = "weather_beta.csv"
file_exists = os.path.isfile(fname)
with open(fname, 'a') as f:
    if file_exists:
        df.to_csv(f, header=False);
    else:
        df.to_csv(f, header=True, columns = columns);
    